In [ ]:
import os
import json
from huggingface_hub import InferenceClient
from dotenv import load_dotenv # dotenv access

load_dotenv() # load .env file
KEY = os.getenv("HF_API_KEY")

In [ ]:
# Defining the models we might use
model_id = "Qwen/Qwen2.5-72B-Instruct"
alt_model_id = 'mistralai/Mistral-7B-Instruct-v0.3'

In [ ]:
# Client intitialization
client = InferenceClient(api_key=KEY, model=model_id)

================================================================================================================================
# Test Code
### Understand the structure of LLM query passing.

In [ ]:
messages = [
	{"role": "user", "content": "Who is the prime minister of India?"}
]
resp = client.chat.completions.create(messages, max_tokens=128)

In [ ]:
print(resp.choices[0].message.content)

================================================================================================================================
# Quiz generation
### Generates quiz MCQs based on the input provided.

In [ ]:
response_json = {
    "1" : {
        "mcq" : "multiple choice question",
        "options" : {
            "A" : "choice here",
            "B" : "choice here",
            "C" : "choice here",
            "D" : "choice here"
        },
        "correct" : "correct answer",
    },
    "2" : {
        "mcq" : "multiple choice question",
        "options" : {
            "A" : "choice here",
            "B" : "choice here",
            "C" : "choice here",
            "D" : "choice here"
        },
        "correct" : "correct answer",
    },
    "3" : {
        "mcq" : "multiple choice question",
        "options" : {
            "A" : "choice here",
            "B" : "choice here",
            "C" : "choice here",
            "D" : "choice here"
        },
        "correct" : "correct answer",
    }
}

In [ ]:
template_generation = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guid. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [ ]:
file_path= r'C:\Hiral\Projects\python randoms\GenAI_Practice\project1\data.txt'
with open(file_path, 'r') as file:
    input_text=file.read()
input_text

In [ ]:
input_number=5
input_subject="Quantum Computing"
input_tone="Professional"
str_json=json.dumps(response_json)

In [ ]:
print(type(str_json), type(response_json))

In [ ]:
def generate_quiz(text, number, subject, tone, response_json):
    formatted_prompt = template_generation.format(
        text=text,
        number=number,
        subject=subject,
        tone=tone,
        response_json = response_json
    )
    messages = [{"role": "user", "content": formatted_prompt}]
    output = client.chat_completion(
        messages=messages,
        model=model_id
    )
    return output.choices[0].message.content

In [ ]:
primary_quiz = generate_quiz(
    text=input_text, 
    number=input_number,
    subject=input_subject, 
    tone=input_tone,
    response_json=str_json
)

print(primary_quiz)

# Generate and Review > independent
### Generates MCQ quiz questions with correct options and reviews the quiz questions and provides updated quiz questions if it doesn't match the complexity for student's subject.

In [ ]:
# Template for reviewing the code
template_review = """
You are an expert english grammarian and writer. Given a MCQ for {subject} students. \
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only \
use at max 50 words for complexity if the quiz is not at par with the cognitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student's ability.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [ ]:
def review_quiz(quiz_text, subject):
    # We inject the output of the first function (quiz_text) into this template
    formatted_prompt = template_review.format(
        subject=subject,
        quiz=quiz_text
    )
    
    messages = [{"role": "user", "content": formatted_prompt}]
    output = client.chat_completion(
        messages=messages,
        model=model_id
    )
    return output.choices[0].message.content

In [ ]:
# --- Reviewing & Refining Quiz ---
reviewed_quiz = review_quiz(
    quiz_text=primary_quiz, # Passing the result of Primary quiz
    subject=input_subject
)

In [ ]:
print("--- Primary Quiz ---")
print(primary_quiz)
print("\n\n--- Reviewed Quiz ---")
print(reviewed_quiz)

# Refined Chain Code

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
llm_client = HuggingFaceEndpoint(
    repo_id=model_id,
    max_new_tokens=2048,
    temperature=0.7,
    huggingfacehub_api_token=KEY
)

In [ ]:
chat_model = ChatHuggingFace(llm=llm_client)

In [ ]:
gen_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=template_generation
)

In [ ]:
review_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=template_review
)

In [ ]:
# Chain 1: Generator
# This chain takes the raw inputs and outputs the Quiz JSON string
quiz_gen_chain = gen_prompt | chat_model | JsonOutputParser()

In [ ]:
# Chain 2: Reviewer
# This chain takes the quiz from Chain 1 and the subject, and outputs the review
review_chain = review_prompt | chat_model | StrOutputParser()

In [ ]:
overall_chain = (
    RunnablePassthrough.assign(quiz=quiz_gen_chain)  # Runs gen_chain, stores result in 'quiz' key
    | RunnablePassthrough.assign(review=review_chain) # Runs review_chain using 'quiz' from prev step
)

In [ ]:
from langchain_core.callbacks import get_usage_metadata_callback

In [ ]:
with get_usage_metadata_callback() as cb:
    result = overall_chain.invoke({
        "text": input_text,
        "number": input_number,
        "subject": input_subject,
        "tone": input_tone,
        "response_json": response_json
    })

In [ ]:
# Prints the usage metadata
print(cb.usage_metadata[model_id])

In [ ]:
result

In [ ]:
quiz = result.get("quiz")
quiz

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

In [ ]:
import pandas as pd
quiz=pd.DataFrame(quiz_table_data)
quiz

In [ ]:
quiz.to_csv("Quantum Computing.csv", index=False)

In [ ]:
print(f"--- Generated Quiz ---\n{result['quiz']}")

In [ ]:
print(f"\n--- Expert Review ---\n{result['review']}")